### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
import time

from epx import Job, ModelConfig, SynthPop

import data_tools as dt

# Simulating a Respiratory Disease Outbreak

### In this notebook, we will model the spread of an influenza through Kewaunee County, WI.

Our influenza model uses an SEIR structure. If you'd like to learn more about disease transmission in FRED, check out Lesson 7 in the Quickstart Guide.

For now, here's a brief overview of the model:
- We begin with 10 agents randomly exposed to the disease. 
- Exposed agents become Infectious after an average of two days, and they can either be Symptomatic (66%) or Asymptomatic (33%). 
- Agents remain infectious for an average of 5 days, during which they can transmit the disease to other susceptible agents that they come into contact with. 
After the infectious period, they recover and are no longer susceptible to reinfection.

In [ ]:
flu_config = ModelConfig(
    synth_pop=SynthPop("US_2010.v5", ["Kewaunee_County_WI"]),
    start_date="2023-01-01",
    end_date="2023-07-01",
)

flu_job = Job(
    "model/main.fred",
    config=[flu_config],
    key="cl_flu_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)

flu_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(flu_job.status) != 'DONE':
    if str(flu_job.status) == 'ERROR':
        logs = flu_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(flu_job.status)

## Exploring the Model Output

In [ ]:
baseline_states = dt.get_states(flu_job)
baseline_exposures = dt.get_explocs(flu_job)
baseline_locations = dt.get_expmap_data(flu_job)

In [ ]:
dt.plot_epicurves(baseline_states)

Our influenza model tracks how many agents are newly exposed, infectious, or recovered each day. In the figure above, we can see that it takes about two weeks from the initial seeding of exposures at the beginning of the simulation for community spread to increase substantially, and that the disease has run its course after about 10 weeks.

Below we plot an animation of infections colored by the type of location at which the exposure occurred (Household, Workplace, etc)  for the duration of the simulation.

In [ ]:
dt.plot_animation_by_exposure_location(baseline_exposures)

Here we show the same data as in the above animation, but summarized into an interactive map with the size of the circles representing households, workplaces, and schools scaled proportionally to the number of agent-to-agent transmission events that took place in those locations throughout the simulation. This helps us visually locate places that experienced unusually high numbers of infections.

In [ ]:
dt.plot_static_exposure_locations(baseline_locations)

Our influenza model also records the demographic characterstics of the agents that were exposed to influenza, so we can segment our exposure data according to those demographic characteristics.

In [ ]:
dt.get_exposure_table_by_demog_group(baseline_exposures)

Here we show another animated map of the same simulation data, this time color-coded by the demographic attributes of the agents that were exposed.

In [ ]:
dt.plot_animation_by_demog_group(baseline_exposures)

The corresponding time series of infections for the above animation is shown below.

In [ ]:
dt.plot_time_series_by_demog_group(baseline_exposures)

In [ ]:
# deleting our job now that we are done with it
flu_job.delete(interactive=False)

FRED is a poweful tool not just for understanding disease spread in the aggregate but also for exploring the individual behaviors that are contributing factors. As part of our baseline model, agents recorded where they were exposed to the respiratory disease.

# Exploring Different Outbreak Scenarios

In [ ]:
from epx import ModelConfigSweep
from itertools import product

We can use our baseline flu model as a jumping off point to explore different outbreak scenarios. That is, we can make different assumptions about the state of the world, people's behavior, etc., and see the effects of those assumptions on our outcomes of interest.

In this case, let's look at three key variables that we suspect will play an important role in determining the size of an outbreak:
1. The transmissibility of the strain of influenza causing the outbreak.
2. The tendency for sick people to isolate themselves (stay home from work or school) while experiencing symptoms.
3. The population's overall sentiment towards getting vaccinated against influenza.

Considering two possibilities for each variable, we have eight total scenarios in which our influenza outbreak might play out very differently:
| Transmissibility | Isolation | Vaccine Sentiment |
|------------------|-----------|-------------------|
| Low              | None      | Negative          |
| Low              | None      | Positive          |
| Low              | Some      | Negative          |
| Low              | Some      | Positive          |
| High             | None      | Negative          |
| High             | None      | Positive          |
| High             | Some      | Negative          |
| High             | Some      | Positive          |

Let's simulate each of these scenarios and compare the results.

In [ ]:
# varying key parameters to explore different scenarios
sweep_params = {
    "transmissiblity_reduction": [0.0, 0.80],
    "stay_home_prob": [0.0, 0.50],
    "willing_to_consider_vaccine": [0.25, 0.75]
}

flu_config_sweep = ModelConfigSweep(
    synth_pop=[SynthPop("US_2010.v5", ["Kewaunee_County_WI"])],
    start_date=["2023-01-01"],
    end_date=["2023-07-01"],
    model_params=[dict(zip(sweep_params, values)) for values in product(*sweep_params.values())]
)

flu_sweep = Job(
    "model/main.fred",
    config=flu_config_sweep,
    key="cl_flu_sweep",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)

flu_sweep.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(flu_sweep.status) != 'DONE':
    if str(flu_sweep.status) == 'ERROR':
        logs = flu_sweep.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(flu_sweep.status)

In [ ]:
high_trans_runs = {
    0: "No Isolation,<br>Negative Vaccine Sentiment",
    1: "No Isolation,<br>Positive Vaccine Sentiment",
    2: "Some Isolation,<br>Negative Vaccine Sentiment",
    3: "Some Isolation,<br>Positive Vaccine Sentiment"
}
low_trans_runs = {
    4: "No Isolation,<br>Negative Vaccine Sentiment",
    5: "No Isolation,<br>Positive Vaccine Sentiment",
    6: "Some Isolation,<br>Negative Vaccine Sentiment",
    7: "Some Isolation,<br>Positive Vaccine Sentiment"
}

low_trans_scenarios, high_trans_scenarios = dt.get_scenario_explocs(flu_sweep, low_trans_runs, high_trans_runs)

The following plot compares the size of the outbreak in the four scenarios where the influenza strain has low transmissibility. 

In [ ]:
dt.plot_epicurve_scenarios(low_trans_scenarios, trans="low")

Although there is random variation between each simulation job, the general tendency is that both isolation and positive vaccine sentiment tend to reduce (and delay) the peak of infections. The effect of having some isolation behavior in the population also tends to be more impactful than the effect of having positive vaccine sentiment, but no isolation behavior. Having both leads to the smallest outbreak among the scenarios.

We can also make a similar plot for the scenarios where the influenza strain has high transmissibility. In those scenarios, the peak of infections tends to be slightly higher than in the analogous scenarios with a low-transmissibility strain, but this effect tends to be less pronounced than the variation in the size of that result from isolation behavior or positive vaccine sentiment being present in the population.

In [ ]:
dt.plot_epicurve_scenarios(high_trans_scenarios, trans="high")

In [ ]:
# deleting our sweep now that we are done with it
flu_sweep.delete(interactive=False)